In [1]:
%load_ext autoreload
%autoreload 2
import sys
import os
current_dir = os.getcwd()
libs_path = os.path.abspath(os.path.join(current_dir, "..", "libs"))
if libs_path not in sys.path:
    sys.path.append(libs_path)

import numpy as np
import pandas as pd
pd.set_option('display.precision', 3)
import data_processing as dp
import models as myML
from typing import List, Optional, Tuple

In [2]:

import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Bidirectional, LSTM, Dense, Flatten, Dropout, RepeatVector, TimeDistributed
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, f1_score
import matplotlib.pyplot as plt
import os
import warnings

warnings.filterwarnings('ignore')
tf.get_logger().setLevel('ERROR')

# Установка seed для воспроизводимости
np.random.seed(1337)
tf.random.set_seed(1337)





dataset = pd.read_excel("../datasets_ref/source.xls")

dataset["temp_Class"] = np.uint(np.bool_(dataset["Class"]))
dataset["Class"] = dataset["temp_Class"]
dataset = dataset.drop(columns=["temp_Class"])

dataset["power, W"] = dataset["Ubs,V"] * dataset["Ibs,A"]
dataset["D+"] = np.sqrt(np.power(dataset["TR11,C"],2) + np.power(dataset["TR13,C"], 2) + np.power(dataset["TR15,C"], 2))
dataset["D-"] = np.sqrt(np.power(dataset["TR12,C"],2) + np.power(dataset["TR14,C"], 2) + np.power(dataset["TR16,C"], 2))
dataset["D"] = np.sqrt(np.power(dataset["D+"],2) + np.power(dataset["D-"], 2))







In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks
from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingClassifier
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
import random
from deap import base, creator, tools, algorithms
import itertools
import warnings
warnings.filterwarnings('ignore')

# ==================== ФИКСАЦИЯ СЛУЧАЙНЫХ ЧИСЕЛ ====================
SEED = 1337
np.random.seed(SEED)
tf.random.set_seed(SEED)
random.seed(SEED)

# ==================== ПАРАМЕТРЫ (ЭМПИРИЧЕСКИ ФИКСИРУЕМ) ====================
WINDOW_SIZE = 10          # размер окна временного ряда
CONV_LAYERS = 2           # число свёрточных слоёв (фиксировано)
RECURRENT_LAYERS = 1      # число рекуррентных слоёв (будет один Bidirectional LSTM)
DENSE_LAYERS = 2          # число полносвязных слоёв в энкодере/декодере
LATENT_DIM = 32           # размерность бутылочного горлышка
BATCH_SIZE = 64
EPOCHS_AUTO = 20         # эпох при поиске гиперпараметров
EPOCHS_FINAL = 50        # эпох при финальном обучении
N_ESTIMATORS = 5         # число моделей в бэггинге

# ==================== ЗАГРУЗКА ДАННЫХ И ФОРМИРОВАНИЕ ОКОН ====================
def load_data(file_path, window_size, has_labels=True, label_col='Class'):
    """
    Загружает CSV, нормализует признаки, создаёт скользящие окна.
    Если есть столбец label_col, возвращает также метки (для классификатора).
    """
    if has_labels:
        df = dataset
    else:
        df = dataset.drop(columns=["Class"])

    
    if has_labels:
        y = df[label_col].values
        X = df.drop(columns=[label_col]).values
    else:
        X = df.values
        y = None

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Формирование окон
    X_windows, y_windows = [], []
    for i in range(len(X_scaled) - window_size + 1):
        X_windows.append(X_scaled[i:i+window_size])
        if y is not None:
            # метка для окна – берём метку последнего момента (или большинство)
            y_windows.append(y[i+window_size-1])
    X_windows = np.array(X_windows)
    y_windows = np.array(y_windows) if y_windows else None
    return X_windows, y_windows, scaler

# ==================== ПОСТРОЕНИЕ АВТОКОДИРОВЩИКА ====================
def create_autoencoder(input_shape, params):
    """
    params: (conv_filters, kernel_size, lstm_units, dense_neurons, optimizer_idx)
    Возвращает модель автокодировщика.
    Архитектура фиксирована: Conv1D x2 -> BiLSTM -> Dense x2 -> Bottleneck -> Dense x2 -> Reshape
    """
    conv_filters, kernel_size, lstm_units, dense_neurons, opt_idx = params
    optimizer_list = ['adam', 'sgd', 'rmsprop', 'adagrad']
    optimizer = optimizer_list[opt_idx]

    # Энкодер
    encoder_input = layers.Input(shape=input_shape)
    x = encoder_input
    for _ in range(CONV_LAYERS):
        x = layers.Conv1D(filters=conv_filters, kernel_size=kernel_size,
                          padding='same', activation='relu')(x)
        x = layers.MaxPooling1D(pool_size=2, padding='same')(x)
    x = layers.Bidirectional(layers.LSTM(lstm_units, return_sequences=False))(x)
    for _ in range(DENSE_LAYERS):
        x = layers.Dense(dense_neurons, activation='relu')(x)
    encoder_output = layers.Dense(LATENT_DIM, activation='relu')(x)

    # Декодер (полносвязный, симметричный)
    d = encoder_output
    for _ in range(DENSE_LAYERS):
        d = layers.Dense(dense_neurons, activation='relu')(d)
    # Восстанавливаем исходную размерность (все временные шаги × признаки)
    decoded = layers.Dense(input_shape[0] * input_shape[1], activation='linear')(d)
    decoder_output = layers.Reshape(input_shape)(decoded)

    autoencoder = models.Model(encoder_input, decoder_output)
    autoencoder.compile(optimizer=optimizer, loss='mse')
    return autoencoder

# ==================== ФУНКЦИЯ ПРИСПОСОБЛЕННОСТИ (ДЛЯ DEAP) ====================
def evaluate_autoencoder(individual, X_train, X_val):
    """Обучает автокодировщик с параметрами individual и возвращает val_loss."""
    params = tuple(individual)
    try:
        input_shape = X_train.shape[1:]  # (window, n_features)
        model = create_autoencoder(input_shape, params)
        history = model.fit(
            X_train, X_train,
            batch_size=BATCH_SIZE,
            epochs=EPOCHS_AUTO,
            validation_data=(X_val, X_val),
            verbose=0,
            callbacks=[callbacks.EarlyStopping(patience=3, restore_best_weights=True)]
        )
        val_loss = min(history.history['val_loss'])
        return val_loss,
    except Exception as e:
        print(f"Ошибка при обучении с params {params}: {e}")
        return 1000.0,   # штраф

# ==================== ГЕНЕТИЧЕСКИЙ АЛГОРИТМ (DEAP) ====================
def setup_deap(X_train, X_val):
    """Настраивает toolbox DEAP для задачи минимизации val_loss."""
    # Пространство поиска
    conv_filters_choices = [32, 64, 128, 256]
    kernel_choices = [3, 5, 7]
    lstm_units_choices = [32, 64, 128, 256]
    dense_neurons_choices = [32, 64, 128, 256]
    optimizer_choices = [0, 1, 2, 3]  # adam, sgd, rmsprop, adagrad

    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMin)

    toolbox = base.Toolbox()
    toolbox.register("attr_conv_filters", random.choice, conv_filters_choices)
    toolbox.register("attr_kernel", random.choice, kernel_choices)
    toolbox.register("attr_lstm_units", random.choice, lstm_units_choices)
    toolbox.register("attr_dense_neurons", random.choice, dense_neurons_choices)
    toolbox.register("attr_optimizer", random.choice, optimizer_choices)

    toolbox.register("individual", tools.initCycle, creator.Individual,
                     (toolbox.attr_conv_filters, toolbox.attr_kernel,
                      toolbox.attr_lstm_units, toolbox.attr_dense_neurons,
                      toolbox.attr_optimizer), n=1)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)

    toolbox.register("mate", tools.cxTwoPoint)
    toolbox.register("mutate", tools.mutUniformInt,
                     low=[32, 3, 32, 32, 0],
                     up=[256, 7, 256, 256, 3],
                     indpb=0.2)
    toolbox.register("select", tools.selTournament, tournsize=3)
    toolbox.register("evaluate", evaluate_autoencoder, X_train=X_train, X_val=X_val)

    return toolbox


from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted


class BiLSTMClassifier(BaseEstimator, ClassifierMixin):
    """Одиночный классификатор BiLSTM + Dense для бэггинга (работает с 3D)."""
    def __init__(self, lstm_units=64, dense_neurons=32, epochs=30, batch_size=64):
        self.lstm_units = lstm_units
        self.dense_neurons = dense_neurons
        self.epochs = epochs
        self.batch_size = batch_size
        self.model = None
        self.classes_ = np.array([0, 1])

    def fit(self, X, y):
        # Разрешаем многомерные входные данные (allow_nd=True)
        X, y = check_X_y(X, y, allow_nd=True, multi_output=False, dtype=np.float32)
        # Если вдруг пришло 2D – превращаем в 3D (один временной шаг)
        if X.ndim == 2:
            X = X.reshape(X.shape[0], 1, -1)
        input_shape = X.shape[1:]
        model = tf.keras.Sequential([
            tf.keras.layers.Bidirectional(
                tf.keras.layers.LSTM(self.lstm_units, return_sequences=False),
                input_shape=input_shape
            ),
            tf.keras.layers.Dense(self.dense_neurons, activation='relu'),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(1, activation='sigmoid')
        ])
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        self.model = model
        self.model.fit(X, y, epochs=self.epochs, batch_size=self.batch_size, verbose=0)
        return self

    def predict(self, X):
        check_is_fitted(self, 'model')
        X = check_array(X, allow_nd=True, dtype=np.float32)
        if X.ndim == 2:
            X = X.reshape(X.shape[0], 1, -1)
        proba = self.model.predict(X)
        return (proba > 0.5).astype(int).flatten()

    def predict_proba(self, X):
        check_is_fitted(self, 'model')
        X = check_array(X, allow_nd=True, dtype=np.float32)
        if X.ndim == 2:
            X = X.reshape(X.shape[0], 1, -1)
        proba = self.model.predict(X)
        return np.hstack([1 - proba, proba])

    def get_params(self, deep=True):
        return {
            'lstm_units': self.lstm_units,
            'dense_neurons': self.dense_neurons,
            'epochs': self.epochs,
            'batch_size': self.batch_size
        }

    def set_params(self, **params):
        for key, value in params.items():
            setattr(self, key, value)
        return self



class BaggingBiLSTMEnsemble:
    """
    Ансамбль из N моделей BiLSTMClassifier, обучаемых на bootstrap-подвыборках.
    """
    def __init__(self, n_estimators=5, lstm_units=64, dense_neurons=32,
                 epochs=30, batch_size=64, random_state=42):
        self.n_estimators = n_estimators
        self.lstm_units = lstm_units
        self.dense_neurons = dense_neurons
        self.epochs = epochs
        self.batch_size = batch_size
        self.random_state = random_state
        self.models = []
        self.classes_ = np.array([0, 1])

    def fit(self, X, y):
        """Обучаем n_estimators моделей на bootstrap-копиях данных."""
        np.random.seed(self.random_state)
        n_samples = X.shape[0]
        self.models = []
        for i in range(self.n_estimators):
            # Генерируем bootstrap-индексы (с повторениями)
            indices = np.random.choice(n_samples, size=n_samples, replace=True)
            X_boot = X[indices]
            y_boot = y[indices]

            model = BiLSTMClassifier(
                lstm_units=self.lstm_units,
                dense_neurons=self.dense_neurons,
                epochs=self.epochs,
                batch_size=self.batch_size
            )
            model.fit(X_boot, y_boot)
            self.models.append(model)
        return self

    def predict_proba(self, X):
        """Усреднённые вероятности по всем моделям ансамбля."""
        probas = np.array([model.predict_proba(X) for model in self.models])
        return np.mean(probas, axis=0)

    def predict(self, X):
        """Метка на основе усреднённой вероятности > 0.5."""
        proba = self.predict_proba(X)[:, 1]  # вероятность класса 1
        return (proba > 0.5).astype(int)


    
def create_bagging_ensemble(X_train, y_train, n_estimators=N_ESTIMATORS):
    """Создаёт и обучает ансамбль BiLSTMClassifier с бэггингом."""
    ensemble = BaggingBiLSTMEnsemble(
        n_estimators=n_estimators,
        lstm_units=64,          
        dense_neurons=32,
        epochs=30,
        batch_size=64,
        random_state=SEED
    )
    ensemble.fit(X_train, y_train)
    return ensemble

# ==================== АНОМАЛИИ ПО АВТОКОДИРОВЩИКУ ====================
def get_anomaly_scores(autoencoder, X, threshold_percentile=95):
    """Вычисляет ошибку реконструкции MSE для каждого окна."""
    reconstructions = autoencoder.predict(X)
    mse = np.mean(np.square(X - reconstructions), axis=(1,2)) #//слабая точка, вместо этой хрени нужно на основе важности + 
    return mse

def detect_anomalies(autoencoder, X_train, X_test, threshold_percentile=95):
    """Определяет порог на обучающей выборке и выдаёт бинарные метки для теста."""
    train_errors = get_anomaly_scores(autoencoder, X_train)
    threshold = np.percentile(train_errors, threshold_percentile)
    test_errors = get_anomaly_scores(autoencoder, X_test)
    preds = (test_errors > threshold).astype(int)
    return preds, threshold

# ==================== ОСНОВНОЙ ПРОЦЕСС ====================
if True:
    data_path_unlabeled = 'unlabeled'
    data_path_labeled = 'labeled'

    """
    data_path_unlabeled: путь к неразмеченному файлу (для обучения автокодировщика)
    data_path_labeled:   путь к размеченному файлу (для обучения и тестирования классификатора)
    """
    # 1. Загружаем неразмеченные данные (только признаки, без 'class')
    X_unlabeled, _, _ = load_data(data_path_unlabeled, WINDOW_SIZE, has_labels=False)

    # Разбиваем на train/val для GA (на неразмеченных данных)
    X_train_auto, X_val_auto = train_test_split(X_unlabeled, test_size=0.2, random_state=SEED)

    # 2. ГЕНЕТИЧЕСКИЙ ПОИСК ГИПЕРПАРАМЕТРОВ АВТОКОДИРОВЩИКА
    print("=== ГЕНЕТИЧЕСКИЙ АЛГОРИТМ (DEAP) ===")
    toolbox = setup_deap(X_train_auto, X_val_auto)
    pop = toolbox.population(n=10)   # размер популяции
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("min", np.min)

    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2,
                                   ngen=5, stats=stats, halloffame=hof,
                                   verbose=True)

    best_params = tuple(hof[0])
    print(f"\nЛучшие гиперпараметры: conv_filters={best_params[0]}, kernel_size={best_params[1]}, "
          f"lstm_units={best_params[2]}, dense_neurons={best_params[3]}, optimizer={best_params[4]}")
    
    # 3. ФИНАЛЬНОЕ ОБУЧЕНИЕ АВТОКОДИРОВЩИКА НА ВСЕХ НЕРАЗМЕЧЕННЫХ ДАННЫХ
    print("\n=== ФИНАЛЬНОЕ ОБУЧЕНИЕ АВТОКОДИРОВЩИКА ===")
    input_shape = X_unlabeled.shape[1:]
    final_autoencoder = create_autoencoder(input_shape, best_params)
    final_autoencoder.fit(
        X_unlabeled, X_unlabeled,
        batch_size=BATCH_SIZE,
        epochs=EPOCHS_FINAL,
        verbose=1,
        callbacks=[callbacks.EarlyStopping(patience=5, restore_best_weights=True)]
    )

    # 4. ЗАГРУЗКА РАЗМЕЧЕННЫХ ДАННЫХ (ДЛЯ КЛАССИФИКАТОРА И ТЕСТИРОВАНИЯ)
    print("\n=== ЗАГРУЗКА РАЗМЕЧЕННЫХ ДАННЫХ ===")
    X_labeled, y_labeled, _ = load_data(data_path_labeled, WINDOW_SIZE, has_labels=True)
    X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(
        X_labeled, y_labeled, test_size=0.3, random_state=SEED, stratify=y_labeled)

    # 5. ОБУЧЕНИЕ БАЗОВОГО КЛАССИФИКАТОРА (БЭГГИНГ BiLSTM)
    print("\n=== ОБУЧЕНИЕ БАЗОВОГО КЛАССИФИКАТОРА (БЭГГИНГ BiLSTM) ===")
    clf = create_bagging_ensemble(X_train_class, y_train_class, n_estimators=N_ESTIMATORS)
    y_pred_class = clf.predict(X_test_class)
    # Предсказания классификатора

    # 6. АНОМАЛИИ ПО АВТОКОДИРОВЩИКУ НА ТЕСТЕ
    print("\n=== ОПРЕДЕЛЕНИЕ АНОМАЛИЙ АВТОКОДИРОВЩИКОМ ===")
    # Используем обучающую выборку классификатора для вычисления порога (она размечена, но метки не используем)
    # Это допустимо – автокодировщик видит только X_train_class (без y)
    y_pred_auto, threshold = detect_anomalies(final_autoencoder, X_train_class, X_test_class, threshold_percentile=95)
    print(f"Порог ошибки (95-й перцентиль на обучении): {threshold:.4f}")

    # 7. СРАВНЕНИЕ РЕЗУЛЬТАТОВ
    print("\n=== СРАВНЕНИЕ МЕТОК: КЛАССИФИКАТОР vs АВТОКОДИРОВЩИК ===")
    print("\n--- Метки классификатора (истинные) ---")
    print(confusion_matrix(y_test_class, y_pred_class))
    print(classification_report(y_test_class, y_pred_class))

    print("\n--- Метки автокодировщика (аномалии) ---")
    print(confusion_matrix(y_test_class, y_pred_auto))
    print(classification_report(y_test_class, y_pred_auto))

    # Дополнительно: процент совпадений меток
    agreement = np.mean(y_pred_class == y_pred_auto)
    print(f"\nПроцент совпадений меток между классификатором и автокодировщиком: {agreement*100:.2f}%")



=== ГЕНЕТИЧЕСКИЙ АЛГОРИТМ (DEAP) ===
gen	nevals	min     
0  	10    	0.254407
1  	9     	0.254407
2  	6     	0.254407
3  	7     	0.254407
4  	6     	0.254407
5  	5     	0.253288

Лучшие гиперпараметры: conv_filters=256, kernel_size=7, lstm_units=128, dense_neurons=64, optimizer=0

=== ФИНАЛЬНОЕ ОБУЧЕНИЕ АВТОКОДИРОВЩИКА ===
Epoch 1/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 0.7869
Epoch 2/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.3215 
Epoch 3/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.3163
Epoch 4/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.3127 
Epoch 5/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.2797
Epoch 6/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.2661 
Epoch 7/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.2587
Epoch 8/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.2527
Epoch 9/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.2458 
Epoch 10/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.2384 
Epoch 11/50
42/

In [4]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, balanced_accuracy_score
bal_acc_clf = balanced_accuracy_score(y_test_class, y_pred_class)
print(f"Balanced Accuracy (классификатор): {bal_acc_clf:.4f}")
# --- Дополнительные метрики для автокодировщика ---
bal_acc_auto = balanced_accuracy_score(y_test_class, y_pred_auto)
print(f"Balanced Accuracy (автокодировщик): {bal_acc_auto:.4f}")



y_pred_proba = clf.predict_proba(X_test_class)[:, 1]

roc_auc_clf = roc_auc_score(y_test_class, y_pred_proba)
print(f"ROC-AUC (классификатор): {roc_auc_clf:.4f}")

anomaly_scores_test=get_anomaly_scores(final_autoencoder, X_test_class)

roc_auc_auto = roc_auc_score(y_test_class, anomaly_scores_test)
print(f"ROC-AUC (автокодировщик, по ошибкам реконструкции): {roc_auc_auto:.4f}")




Balanced Accuracy (классификатор): 0.9508
Balanced Accuracy (автокодировщик): 0.6499
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
ROC-AUC (классификатор): 0.9972
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
ROC-AUC (автокодировщик, по ошибкам реконструкции): 0.9182


In [ ]:
def get_anomaly_scores2(autoencoder, X, threshold_percentile=95):
    """Вычисляет ошибку реконструкции MSE для каждого окна."""
    reconstructions = autoencoder.predict(X)
    mse = np.mean(np.square(X - reconstructions), axis=(1,2))
    return reconstructions

In [ ]:
get_anomaly_scores2(final_autoencoder, X_train_class).shape